# Entwicklung eines Programms zum Lesen einer Endspiel-Stellung und Spielen mit Hilfe der Endgame-Database

In [1]:
import chess
import chess.gaviota
from IPython.display import display, clear_output

In [2]:
class Checkmate(Exception): pass
class Stalemate(Exception): pass

In [3]:
def play(chess_position: str) -> None:
    
    # Try-Block to intercept checkmate or stalemate
    try:

        # Init board with given endgame position
        board = chess.Board(chess_position)

        # Open Gaviota tablebase to determine automatic moves
        with chess.gaviota.open_tablebase("./gaviota/gtb/gtb4") as tablebase:

            #Loop for playing against the automatic moves
            while True:
                display(board)
                board = move_white(board)
                if board.is_checkmate(): raise Checkmate
                if board.is_stalemate(): raise Stalemate
                display(board)
                board = move_black(board, tablebase)
                if board.is_checkmate(): raise Checkmate
                if board.is_stalemate(): raise Stalemate

    # Exception if board is checkmate
    except Checkmate:
        display(board)
        print("Checkmate")

    # Exception if board is stalemate
    except Stalemate:
        display(board)
        print("Stalemate")

In [4]:
def move_white(board: chess.Board) -> chess.Board:
    'Asks the player for a legal move for white and executes it.'

    while True:
        print("Please insert your move in UCI format (e.g. a1a2):")
        move_uci = input()
        try: 
            move = chess.Move.from_uci(move_uci)
            if move in board.legal_moves:
                board.push(move)
                return board
            else:
                print(move_uci, "is not a valid move.")
        except ValueError:
            print(move_uci, "is not valid UCI format.")

In [5]:
def move_black(board: chess.Board, tablebase: chess.gaviota.NativeTablebase) -> chess.Board:
    'Executes the best depth-to-mate move for black.'

    # Get first legal move with dtm
    legal_moves = board.generate_legal_moves()
    black_move = next(legal_moves)
    board.push(black_move)
    best_move = (black_move, tablebase.probe_dtm(board))
    board.pop()

    # Loop through legal moves, update best move using dtm
    for black_move in legal_moves:
        board.push(black_move)
        if board.is_checkmate(): raise Checkmate
        curr_dtm = tablebase.probe_dtm(board)
        if is_better_move(curr_dtm, best_move[1]):
            best_move = (black_move, curr_dtm)
        board.pop()

    board.push(best_move[0])
    return board

In [6]:
def is_better_move(new: int, old: int) -> bool:
    "Takes two depth-to-mates and determines if first one is better for black."

    if old > 0:
        if new > 0:
            return new > old
        elif new < 0:
            return True
        else:
            return True
    elif old < 0:
        if new > 0:
            return False
        elif new < 0:
            return new > old
        else:
            return False
    else:
        if new > 0:
            return False
        elif new < 0:
            return True
        else:
            return False


In [7]:
play("8/8/8/8/8/8/8/K2kr3 w - - 0 1")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6e34dfb-c85e-40db-bde6-d0ca8b0148c0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>